# Finance Decision Analytics Project
## Cash Buffer and Liquidity Risk (SQL + Python)

### Project Title
Finance Decision Analytics: Cash Buffer and Liquidity Risk Monitoring

### Problem Context
Small businesses often fail to notice liquidity risk until it becomes urgent.
This project builds a reliable data pipeline and decision focused analysis to measure cash buffer strength, identify drawdowns, and test scenarios that impact financial stability.

### Key Questions (Decision Focused)
1. What is the current cash buffer level and how many days of operations can the business sustain under normal conditions
2. What is the worst drawdown behavior and how quickly does cash recover after a shock
3. What cash level thresholds should be considered safe, caution, and critical using percentiles and drawdown history
4. Which days or weeks have the highest liquidity risk based on cashflow volatility and recurring obligations
5. What happens to liquidity risk if revenue drops, expenses rise, or payroll timing shifts (scenario impact)

### Goals
- Build a clean, reproducible pipeline from raw data to decision ready tables
- Create a daily cash balance and cashflow view that is trustworthy
- Quantify liquidity risk using drawdown and percentile based thresholds
- Produce practical recommendations that a business owner can act on
- Provide scenario analysis to stress test liquidity under realistic changes

### Tools and Technologies
- MySQL (data modelling, staging, standardisation, fact tables, daily aggregates)
- Python (analysis, scenario simulation, decision metrics)
- pandas, numpy (data handling and calculations)
- matplotlib (optional charts for internal validation)

### High Level Workflow
1. Data ingestion and staging (stg tables)
   - stg_checking_main
   - stg_checking_secondary
   - stg_credit_card
   - stg_payroll
   - stg_payroll_bc

2. Standardisation (std tables)
   - Standardise column names, data types, categories, and transaction direction
   - Validate amounts, dates, and duplicates using clear rules

3. Core modelling
   - Build fact_transactions as the single source of truth for transactions
   - Create daily_cashflow to summarise inflows and outflows by day
   - Create daily_cash_balance to compute rolling balance by day

4. Cash buffer analysis (decision layer)
   - Compute buffer distribution and key percentiles
   - Compute drawdowns, peak to trough declines, and recovery behaviour
   - Define safe, caution, and critical thresholds

5. Scenario testing (decision layer)
   - Revenue drop scenarios
   - Expense increase scenarios
   - Payroll timing shifts
   - Compare outcomes vs baseline and quantify risk change

6. Outputs
   - Decision ready metrics and thresholds
   - Simple narrative recommendations
   - Reproducible SQL queries and Python script for reruns


# Liberaries

In [4]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql

# Database Connection
## Connecting to MySQL for Cash Buffer Analysis

In this section, we establish a secure connection to the MySQL database
that contains the final analytical tables.

Only read operations are performed in this notebook.
All data modelling and transformations were completed upstream in SQL.

The purpose of this step is to load decision ready tables
for liquidity and cash buffer analysis.


In [10]:
from sqlalchemy import create_engine

MYSQL_USER = "root"
MYSQL_PASSWORD = "82476737"
MYSQL_HOST = "localhost"
MYSQL_PORT = 3306
MYSQL_DB = "finance_decision_analytics"



engine = create_engine(
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
)

with engine.connect() as conn:
    result = conn.exec_driver_sql("SELECT 1;").fetchall()
    print(result)



[(1,)]


## Load Analytical Tables
Loading decision-ready tables from MySQL for cash buffer analysis


In [13]:
df_balance = pd.read_sql(
    "SELECT * FROM daily_cash_balance ORDER BY txn_date;",
    engine
)

df_balance.head()



,txn_date,cash_in,cash_out,net_flow,cumulative_balance
0,2022-01-01,4205.0,0.0,4205.0,4205.0
1,2022-01-02,970.0,0.0,970.0,5175.0
2,2022-01-03,888.0,0.0,888.0,6063.0
3,2022-01-04,821.0,430.0,391.0,6454.0
4,2022-01-05,787.0,0.0,787.0,7241.0


## Data Validation and Integrity Checks

Before proceeding to any liquidity or cash buffer analysis, 
the daily cash balance data is validated to ensure analytical reliability.

The following checks are performed:
- Convert transaction dates to proper datetime format
- Ensure records are ordered chronologically
- Verify that each date appears only once (daily granularity)
- Confirm there are no missing dates in the time series
- Validate that cumulative balance is mathematically consistent with net cash flow

These checks ensure that all downstream risk metrics and drawdown calculations
are based on clean, trustworthy, and decision-ready data.


In [14]:
df_balance.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   txn_date            730 non-null    object 
 1   cash_in             730 non-null    float64
 2   cash_out            730 non-null    float64
 3   net_flow            730 non-null    float64
 4   cumulative_balance  730 non-null    float64
dtypes: float64(4), object(1)
memory usage: 28.6+ KB


In [15]:
df_balance["txn_date"] = pd.to_datetime(df_balance["txn_date"])

In [ ]:
df_balance = df_balance.sort_values("txn_date").reset_index(drop=True)

In [17]:
df_balance["txn_date"].is_unique

True

In [18]:
(df_balance["net_flow"].cumsum() == df_balance["cumulative_balance"]).all()

True

## Cash Buffer and Drawdown Analysis

In this section, we quantify liquidity risk by analysing cash buffer behaviour
over time.

The analysis focuses on:
- Identifying peak-to-trough drawdowns in cumulative cash balance
- Measuring drawdown depth and recovery behaviour
- Establishing historical risk patterns that inform safe, caution, and critical thresholds

The goal is not prediction, but understanding downside risk
and resilience based on observed cash balance dynamics.


In [19]:
# Ensure chronological order
df_balance = df_balance.sort_values("txn_date").reset_index(drop=True)


In [20]:
# Rolling peak of cumulative balance
df_balance["rolling_peak"] = df_balance["cumulative_balance"].cummax()

# Drawdown amount
df_balance["drawdown"] = (
    df_balance["cumulative_balance"] - df_balance["rolling_peak"]
)


In [21]:
# Percentage drawdown
df_balance["drawdown_pct"] = (
    df_balance["drawdown"] / df_balance["rolling_peak"]
)

In [22]:
df_balance["drawdown"].min(), df_balance["drawdown_pct"].min()

(-4673.0, -0.11380062730124096)

## Interpretation and Decision Insight

The historical drawdown analysis shows that the business has not experienced
a liquidity collapse and demonstrates a reasonable level of resilience
under past cash flow fluctuations.

However, the observed drawdown depth indicates that the cash buffer margin
is relatively limited. While past shocks were absorbed, the system operates
with a constrained safety cushion.

If similar drawdowns were to become deeper or persist for longer periods,
liquidity risk would increase materially and could threaten operational stability.

This analysis highlights the importance of monitoring downside risk
and proactively managing cash buffer thresholds,
transforming raw balance data into decision-grade financial insight.


---

## Liquidity Risk Thresholds

To translate historical drawdown behaviour into actionable decisions,
we define liquidity risk thresholds based on the empirical distribution
of percentage drawdowns.

These thresholds classify cash buffer conditions into:
- Safe: normal historical fluctuations
- Caution: elevated but manageable liquidity stress
- Critical: extreme downside risk requiring immediate attention

Thresholds are derived from historical percentiles rather than arbitrary rules.


In [34]:
# Only drawdown days for threshold estimation
dd = df_balance.loc[df_balance["drawdown_pct"] < 0, "drawdown_pct"]

# Thresholds (more negative = higher risk)
caution_th = dd.quantile(0.20)   # worse 20% of drawdown days
critical_th = dd.quantile(0.05)  # worst 5% of drawdown days

# Default
df_balance["liquidity_risk"] = "Safe"

# Apply buckets
mask = df_balance["drawdown_pct"] < 0

df_balance.loc[mask & (df_balance["drawdown_pct"] <= caution_th), "liquidity_risk"] = "Caution"
df_balance.loc[mask & (df_balance["drawdown_pct"] <= critical_th), "liquidity_risk"] = "Critical"

caution_th, critical_th


(-0.020880899373101964, -0.04976164657114759)

In [35]:
df_balance["liquidity_risk"].value_counts()

liquidity_risk
Safe        704
Caution      19
Critical      7
Name: count, dtype: int64

In [36]:
df_balance.sort_values("drawdown_pct").head(15)[
    ["txn_date", "cumulative_balance", "rolling_peak", "drawdown", "drawdown_pct", "liquidity_risk"]
]


,txn_date,cumulative_balance,rolling_peak,drawdown,drawdown_pct,liquidity_risk
14,2022-01-15,12997.0,14666.0,-1669.0,-0.113801,Critical
45,2022-02-15,40431.0,44849.0,-4418.0,-0.098508,Critical
46,2022-02-16,41344.0,44849.0,-3505.0,-0.078151,Critical
73,2022-03-15,60034.0,64630.0,-4596.0,-0.071112,Critical
74,2022-03-16,60751.0,64630.0,-3879.0,-0.060019,Critical
75,2022-03-17,61202.0,64630.0,-3428.0,-0.053040,Critical
47,2022-02-17,42492.0,44849.0,-2357.0,-0.052554,Critical
104,2022-04-15,78019.0,81659.0,-3640.0,-0.044576,Caution
76,2022-03-18,61896.0,64630.0,-2734.0,-0.042302,Caution
105,2022-04-16,78331.0,81659.0,-3328.0,-0.040755,Caution


## Risk Summary for Decision Use

After defining liquidity risk thresholds from historical drawdown behaviour,
we summarise how often the business enters stress conditions.

This converts raw balance movements into a practical monitoring view:
how many days were normal (Safe), elevated risk (Caution), and tail-risk stress (Critical).


In [37]:
risk_summary = (
    df_balance["liquidity_risk"]
    .value_counts()
    .rename_axis("risk_level")
    .reset_index(name="days")
)

risk_summary["share_pct"] = (risk_summary["days"] / len(df_balance) * 100).round(2)
risk_summary


,risk_level,days,share_pct
0,Safe,704,96.44
1,Caution,19,2.60
2,Critical,7,0.96


In [38]:
df_balance.loc[df_balance["liquidity_risk"] == "Critical", [
    "txn_date", "cash_in", "cash_out", "net_flow", "cumulative_balance", "drawdown_pct"
]].sort_values("txn_date")


,txn_date,cash_in,cash_out,net_flow,cumulative_balance,drawdown_pct
14,2022-01-15,1019.0,2688.0,-1669.0,12997.0,-0.113801
45,2022-02-15,798.0,5216.0,-4418.0,40431.0,-0.098508
46,2022-02-16,913.0,0.0,913.0,41344.0,-0.078151
47,2022-02-17,1148.0,0.0,1148.0,42492.0,-0.052554
73,2022-03-15,632.0,5228.0,-4596.0,60034.0,-0.071112
74,2022-03-16,717.0,0.0,717.0,60751.0,-0.060019
75,2022-03-17,451.0,0.0,451.0,61202.0,-0.053040


In [41]:
export_cols = [
    "txn_date",
    "cash_in",
    "cash_out",
    "net_flow",
    "cumulative_balance",
    "rolling_peak",
    "drawdown",
    "drawdown_pct",
    "liquidity_risk"
]

df_balance[export_cols].to_csv(
    "E:\\Project\\GITHub\\Project03\\powerbi\\daily_cash_buffer.csv",
    index=False
)

risk_summary.to_csv(
    "E:\\Project\\GITHub\\Project03\\powerbi\\liquidity_risk_summary.csv",
    index=False
)
